In [2]:
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np

In [77]:
x,y=make_classification(n_samples=240,
    n_features=3,         
    n_informative=2,      
    n_redundant=0,        
    n_repeated=0,         
    n_classes=3,          
    n_clusters_per_class=1)

In [78]:
x.shape

(240, 3)

In [79]:
data=pd.DataFrame(x,columns=[f'f{i}' for i in range(x.shape[1])])
data['target']=y

In [80]:
data

,f0,f1,f2,target
0,-1.711415,0.660793,0.554897,0
1,-1.188883,1.050261,-1.168095,0
2,-0.987773,-1.107063,0.774159,2
3,-1.140649,2.060957,-0.202325,0
4,-0.427471,-1.183106,0.513062,1
...,...,...,...,...
235,-0.319603,-0.886994,1.232561,1
236,2.212901,-0.970272,0.511359,2
237,-0.699166,1.363681,-1.716893,0
238,0.866972,0.644451,1.570905,1


In [81]:
data['target']=data['target'].astype('category')

In [82]:
data=pd.get_dummies(data)
data

,f0,f1,f2,target_0,target_1,target_2
0,-1.711415,0.660793,0.554897,True,False,False
1,-1.188883,1.050261,-1.168095,True,False,False
2,-0.987773,-1.107063,0.774159,False,False,True
3,-1.140649,2.060957,-0.202325,True,False,False
4,-0.427471,-1.183106,0.513062,False,True,False
...,...,...,...,...,...,...
235,-0.319603,-0.886994,1.232561,False,True,False
236,2.212901,-0.970272,0.511359,False,False,True
237,-0.699166,1.363681,-1.716893,True,False,False
238,0.866972,0.644451,1.570905,False,True,False


In [83]:
data[['target_0','target_1','target_2']]=data[['target_0','target_1','target_2']].astype(int)
data

,f0,f1,f2,target_0,target_1,target_2
0,-1.711415,0.660793,0.554897,1,0,0
1,-1.188883,1.050261,-1.168095,1,0,0
2,-0.987773,-1.107063,0.774159,0,0,1
3,-1.140649,2.060957,-0.202325,1,0,0
4,-0.427471,-1.183106,0.513062,0,1,0
...,...,...,...,...,...,...
235,-0.319603,-0.886994,1.232561,0,1,0
236,2.212901,-0.970272,0.511359,0,0,1
237,-0.699166,1.363681,-1.716893,1,0,0
238,0.866972,0.644451,1.570905,0,1,0


In [84]:
y=pd.concat([data['target_0'],data['target_1'],data['target_2']],axis=1)

In [85]:
y

,target_0,target_1,target_2
0,1,0,0
1,1,0,0
2,0,0,1
3,1,0,0
4,0,1,0
...,...,...,...
235,0,1,0
236,0,0,1
237,1,0,0
238,0,1,0


In [86]:
from sklearn.model_selection import train_test_split

In [87]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2)

In [88]:
class softmax_regression():
    
    def __init__(self,learning_rate=0.01,epochs=100):
        self.m=None #Weight
        self.b=None #Bias
        self.lr=learning_rate
        self.epochs=epochs
        
    def softmax(self,z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def compute_loss(self, y_true, y_pred):
        return -np.mean(np.sum(y_true * np.log(y_pred + 1e-15), axis=1))
    
    def fit(self,xtrain,ytrain):
        n_samples,n_features=xtrain.shape
        n_classes=ytrain.shape[1]
        
        self.m=np.ones((n_features,n_classes))
        self.b=np.ones((1,n_classes))
        
        for epoch in range(self.epochs):
            z=np.dot(xtrain,self.m) + self.b
            y_pred=self.softmax(z)
            
            error=y_pred-ytrain
            
            L=self.compute_loss(ytrain,y_pred)
            print(f"Epoch {epoch+1}/{self.epochs}, Loss: {L:.4f}")
            
            dL_dm = np.dot(xtrain.T, error) / n_samples
            dL_db = np.sum(error.to_numpy(), axis=0, keepdims=True) / n_samples
#             dL_db = error.sum(axis=0) / n_samples
            
            self.m= self.m - self.lr * dL_dm
            self.b= self.b - self.lr * dL_db
        return self.m,self.b
    
    def predict(self,xtest):
        z=np.dot(xtest,self.m) + self.b
        y_pred=self.softmax(z)
        return np.argmax(y_pred,axis=1)

In [112]:
softmax_reg=softmax_regression(learning_rate=0.1)
softmax_reg.fit(xtrain,ytrain)

Epoch 1/100, Loss: 1.0986
Epoch 2/100, Loss: 1.0435
Epoch 3/100, Loss: 0.9952
Epoch 4/100, Loss: 0.9528
Epoch 5/100, Loss: 0.9157
Epoch 6/100, Loss: 0.8832
Epoch 7/100, Loss: 0.8546
Epoch 8/100, Loss: 0.8294
Epoch 9/100, Loss: 0.8071
Epoch 10/100, Loss: 0.7872
Epoch 11/100, Loss: 0.7695
Epoch 12/100, Loss: 0.7536
Epoch 13/100, Loss: 0.7393
Epoch 14/100, Loss: 0.7264
Epoch 15/100, Loss: 0.7146
Epoch 16/100, Loss: 0.7039
Epoch 17/100, Loss: 0.6941
Epoch 18/100, Loss: 0.6851
Epoch 19/100, Loss: 0.6768
Epoch 20/100, Loss: 0.6691
Epoch 21/100, Loss: 0.6620
Epoch 22/100, Loss: 0.6554
Epoch 23/100, Loss: 0.6493
Epoch 24/100, Loss: 0.6436
Epoch 25/100, Loss: 0.6382
Epoch 26/100, Loss: 0.6331
Epoch 27/100, Loss: 0.6284
Epoch 28/100, Loss: 0.6239
Epoch 29/100, Loss: 0.6197
Epoch 30/100, Loss: 0.6157
Epoch 31/100, Loss: 0.6120
Epoch 32/100, Loss: 0.6084
Epoch 33/100, Loss: 0.6050
Epoch 34/100, Loss: 0.6017
Epoch 35/100, Loss: 0.5987
Epoch 36/100, Loss: 0.5957
Epoch 37/100, Loss: 0.5929
Epoch 38/1

(array([[0.32614976, 0.66481954, 2.0090307 ],
        [2.12745496, 0.53216831, 0.34037673],
        [0.97003597, 1.05436126, 0.97560277]]),
 array([[0.89432149, 1.04419928, 1.06147923]]))

In [113]:
ypred=softmax_reg.predict(xtest)
# ypred = np.argmax(ypred, axis=1)
ypred

array([0, 2, 2, 1, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0,
       2, 2, 1, 2, 2, 0, 2, 1, 0, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 0,
       0, 0, 0, 0], dtype=int64)

In [114]:
ytest_class_labels = np.argmax(ytest, axis=1)

In [115]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(ypred,ytest_class_labels)

0.8333333333333334

In [116]:
confusion_matrix(ypred,ytest_class_labels)

array([[19,  0,  0],
       [ 2,  8,  5],
       [ 0,  1, 13]], dtype=int64)